In [1]:
import numpy as np 
import pandas as pd

In [2]:
diabetes_df = pd.read_csv("diabetes.csv")
diabetes_df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [3]:
from sklearn.linear_model import Ridge, LinearRegression, Lasso, LogisticRegression
from sklearn.model_selection import KFold, train_test_split, cross_val_score
from sklearn.metrics import mean_squared_error, root_mean_squared_error

X = diabetes_df.drop(["Outcome"], axis=1).values
y = diabetes_df['Outcome'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.3, random_state=69)

X_train.shape, y_train.shape, X_test.shape, y_test.shape


((537, 8), (537,), (231, 8), (231,))

In [4]:
reg = LinearRegression()
reg.fit(X_train, y_train)
y_pred = reg.predict(X_test)
r_squard = reg.score(X_test, y_test)
mse = mean_squared_error(y_test, y_pred)
rmse = root_mean_squared_error(y_test, y_pred)
print(r_squard)
print("mse: " + str(mse) + "\nrmse: " + str(rmse))

0.3046721986545401
mse: 0.16006834039331402
rmse: 0.4000854163716968


In [5]:
alphas = [0.0001, 0.1, 1, 10, 100]
scores = []
for alpha in alphas:
    reg = Ridge(alpha = alpha)
    reg.fit(X_train, y_train)
    score = reg.score(X_test, y_test)
    scores.append(score)
print(scores)

[0.3046721717759914, 0.30464535716466046, 0.30440708054714505, 0.3023133696328062, 0.29359436413074225]


In [9]:
reg = LogisticRegression()

reg.fit(X_train,y_train)

y_pred = reg.predict_proba(X_test)[:, 1]
print(y_pred)

[0.11771431 0.77549035 0.27816808 0.39675115 0.15157928 0.75182405
 0.06319721 0.03329957 0.07555403 0.01853515 0.06286096 0.18563421
 0.23715789 0.13831684 0.68321589 0.38219426 0.3272368  0.50085509
 0.53855248 0.04806141 0.27682182 0.23213483 0.48703445 0.20503902
 0.08347585 0.27924163 0.61549893 0.22036174 0.42998142 0.34011062
 0.14213675 0.91940396 0.34447468 0.23211298 0.39989091 0.05685349
 0.0913342  0.63456549 0.05084158 0.89869468 0.09598654 0.6005205
 0.6131067  0.62803648 0.0332886  0.62200797 0.38328384 0.66485036
 0.08818819 0.37503079 0.038759   0.04767695 0.84262725 0.57227022
 0.67772214 0.27751685 0.72837951 0.11047887 0.05160258 0.77752727
 0.44623818 0.63240888 0.45957447 0.21546555 0.29661266 0.17918382
 0.86273983 0.21263826 0.20231543 0.03819874 0.15885349 0.19857904
 0.17885591 0.21881069 0.68290382 0.40658428 0.81167237 0.87168136
 0.24600403 0.45780077 0.29420412 0.7909454  0.8057884  0.0894401
 0.76201036 0.08543618 0.67030044 0.18343218 0.58349679 0.050040